# Description of what this code does needs to go here
*Question: Traveltimes or travelspeeds?*

Logger for writing outputs to .log file.
.info prints to standard output + .log file
.debug prints to only .log file

In [1]:
import logging
import time
import sys
logdatetime = time.strftime("%m%d")

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.handlers =[]

fh = logging.FileHandler('test'+logdatetime+'.log', mode='w')
fh.setLevel(logging.DEBUG)

sh = logging.StreamHandler(stream=sys.stdout)
sh.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)

logger.addHandler(fh)
logger.addHandler(sh)

In [2]:
import config

'''Select path name for files depending on whether you are working on your PC or cluster'''
# path_for_data is an optional variable that defaults to 'path'. This is included to facilate a seperate path
# address for large data files

path = './Data_Files/'         # Use this if working on PC
path_for_data = '../../'       # Use this if working on PC and if data files are in a different folder

#path = './scratch/'           # Use this if working on cluster
#path_for_data = './scratch/'  # Use this if working on cluster and if data files are in a different folder

filenames = config.generate_filenames(path, path_for_data)

logger.info('Filenames have been loaded')
[logger.debug(i) for i in filenames]

logger.info('Constants have been loaded')
[logger.debug(i) for i in dir(globals()['config'])];

Filenames have been loaded
Constants have been loaded


In [3]:
import numpy as np
import pandas as pd
from read_data import read_full_link_json
import Phase1      # This name needs to be changed
%matplotlib inline

full_link_ids, D = read_full_link_json(filenames)  # Reads trips or travelspeeds depending on config.TRIPS value 
D = pd.DataFrame(D)

logger.info('Full_link data has been read')

if config.SEEDED == 1:
    seed_W = 0
    seed_H = 1
elif config.SEEDED == 0:
    seed_W = None
    seed_H = None
else:
    logger.critical('Seed value invalid. Needs to be 0 or 1. Check config.py!')
    quit()

W, H, results = Phase1.SNMF(D, filenames, rank = config.RANK, beta = 0.00001, seed_W = seed_W, seed_H = seed_H)

Trips
Full_link data has been read
Rank= 50, Beta= 1e-05, Threshold= 0.2
Initializing W and H...
W, H chosen
Iteration= 0, Error= 49.7183895477
Iteration= 1, Error= 48.0324813424
Iteration= 2, Error= 48.0031085544
Iteration= 3, Error= 47.9671853406
Iteration= 4, Error= 47.9256330076
Iteration= 5, Error= 47.8791742078
Iteration= 6, Error= 47.8263696163
Iteration= 7, Error= 47.7646389608
Iteration= 8, Error= 47.6903686639
Iteration= 9, Error= 47.5987694462
Iteration= 10, Error= 47.4836725674
Iteration= 11, Error= 47.3373884268
Iteration= 12, Error= 47.1507936742
Iteration= 13, Error= 46.9138909245
Iteration= 14, Error= 46.6171299434
Iteration= 15, Error= 46.2536582246
Iteration= 16, Error= 45.8222119034
Iteration= 17, Error= 45.3295717476
Iteration= 18, Error= 44.7909325929
Iteration= 19, Error= 44.2271304373
Iteration= 20, Error= 43.6596113141
Iteration= 21, Error= 43.1056813594
Iteration= 22, Error= 42.5761555959
Iteration= 23, Error= 42.0755992851
Iteration= 24, Error= 41.6039850575
I

KeyboardInterrupt: 

In [ ]:
results


In [ ]:
if config.TRIPS == 1:
    W.to_csv(filenames['W_trips']);                     print('W_trips written')
    H.to_csv(filenames['H_trips']);                     print('H_trips written')
    Phase1.axe_H(H).to_csv(filenames['H_trips_axed']);  print('H_trips_axed written')

elif config.TRIPS == 0:
    W.to_csv(filenames['W_speeds']);                     print('W_speeds written')
    H.to_csv(filenames['H_speeds']);                     print('H_speeds written')
    Phase1.axe_H(H).to_csv(filenames['H_speeds_axed']);  print('H_speeds_axed written')

In [ ]:
np.sum(W)

In [ ]:
results.plot(y='error')
results.plot(y='sparsity')
results.plot(y='H_min')

In [ ]:
H_loaded = pd.read_csv(filenames['H_trips'], names=['orig_index']+full_link_ids, skiprows=1)
H_loaded
sig0_link_ids = H_loaded.loc[0,:][H_loaded.loc[0,:]>0].keys().tolist()[1:]
sig0_link_ids

In [ ]:
import geopandas as gpd
from shapely.geometry import LineString

df = gpd.pd.read_csv(filenames['links'])
df = df[df['link_id'].isin(sig0_link_ids)]
geometry = [LineString([(x1,y1),(x2,y2)]) for (x1,y1,x2,y2) in zip(df.startX, df.startY, df.endX, df.endY)]

crs = {'init': 'epsg:4326'}
geo_df = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
geo_df
geo_df.to_file('sig0')